# Uniaxial Channel Waveguide (Rotated)

This example extends the uniaxial channel waveguide problem (`uniaxial_channel.ipynb`) by rotating the extraordinary (c-) axis of the uniaxial core material by 45 degrees within the xy plane. When the c-axis is rotated, the permittivity tensor remains symmetric but is no longer diagonal, resulting in non-zero off-diagonal components epsxy and epsyx. This example demonstrates how the solver handles anisotropic materials in arbitrary orientations and how the mesh-building tools accommodate permittivity tensors that include coupling between field components.

As in the previous example, we simulate the full waveguide cross-section without imposing symmetry conditions in x or y. In the rotated case, symmetry reduction is not possible: the presence of off-diagonal permittivity terms breaks the mirror symmetry that would otherwise allow the computational domain to be halved (or quartered). Using the entire domain ensures the correct representation of the rotated anisotropy and allows direct comparison with the unrotated example.

In [ ]:
# Enable automatic reloading of modules (IPython only)
%reload_ext autoreload
%autoreload 2 

# load required modules
import modesolver as mode
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [ ]:
# Refractive indices:
n1 = 1.55           # cladding index
n2x = 2.156         # extraordinary index (core)
n2y = 2.232         # ordinary index (core)
n2z = 2.232         # ordinary index (core)
theta = -np.pi/4     # orientation of extraordinary axis

e2xx = n2x**2 * np.cos(theta)**2 + n2y**2 * np.sin(theta)**2
e2yy = n2y**2 * np.cos(theta)**2 + n2x**2 * np.sin(theta)**2
e2xy = np.cos(theta) * np.sin(theta) * (n2x**2 - n2y**2)
e2yx = e2xy

# channel waveguide dimensions
Rx = 0.30           # half-width in x
Ry = 0.20           # half-height in y
side = 0.2          # cladding thickness (to be stretched by 4X later)

dx = 0.0025         # grid size (x)
dy = dx             # grid size (y)

# Build waveguide mesh (epsxx, epsxy, epsyx, epsyy, epszz)
x,y,xc,yc,nx,ny,epsxx       = mode.waveguidemeshfull([n1,np.sqrt(e2xx),n1], [side,2*Ry,side], 2*Ry, Rx, side, dx, dy)
x,y,xc,yc,nx,ny,epsxy       = mode.waveguidemeshfull([0, np.sqrt(e2xy.astype(complex)),0 ], [side,2*Ry,side], 2*Ry, Rx, side, dx, dy)
x,y,xc,yc,nx,ny,epsyx       = mode.waveguidemeshfull([0, np.sqrt(e2yx.astype(complex)),0 ], [side,2*Ry,side], 2*Ry, Rx, side, dx, dy)
x,y,xc,yc,nx,ny,epsyy       = mode.waveguidemeshfull([n1,np.sqrt(e2yy),n1], [side,2*Ry,side], 2*Ry, Rx, side, dx, dy)
x,y,xc,yc,nx,ny,epszz,edges = mode.waveguidemeshfull([n1,n2z,n1],           [side,2*Ry,side], 2*Ry, Rx, side, dx, dy, return_edges=True)

In [ ]:
# Stretch cladding mesh on all four boundaries by 4X (from 0.2 µm to 0.8 µm):
x,y,xc,yc,dx,dy = mode.stretchmesh(x,y,[80,80,80,80],[4,4,4,4])

In [ ]:
wavelength = 1.0   # vacuum wavelength
nmodes = 2         # here we calculate two modes

neff, _, _, _, Hx, Hy, Hz = mode.wgmodes(wavelength, n2y, nmodes, dx, dy, epsxx=epsxx, epsxy=epsxy, epsyx = epsyx, epsyy=epsyy, epszz=epszz, boundary='0000')
print("neff =", [f"{v:.6f}" for v in neff])

In [ ]:
# Plot Modes
levels = np.arange(-45,0,3)
fig, axes = plt.subplots(nmodes,2, figsize=(8,8))

for m in range(nmodes):
    axHx = axes[m,0]
    axHy = axes[m,1]

    axHx.pcolormesh(x,y,abs(Hx[:,:,m]),cmap='jet',vmax=1)
    axHx.contour(x,y,20*np.log10(abs(Hx[:,:,m])),levels,colors='white',negative_linestyles='solid', linewidths=1)
    axHx.set_aspect('equal')
    axHx.axis([min(x), max(x), min(y), max(y)])
    axHx.set_xlabel('x')
    axHx.set_ylabel('y')
    axHx.set_title(f'Hx (mode {m})')
    l = LineCollection(edges, colors='black', linewidths=1)
    axHx.add_collection(l)

    axHy.pcolormesh(x,y,abs(Hy[:,:,m]),cmap='jet',vmax=1)
    axHy.contour(x,y,20*np.log10(abs(Hy[:,:,m])),levels,colors='white',negative_linestyles='solid', linewidths=1)
    axHy.set_aspect('equal')
    axHy.axis([min(x), max(x), min(y), max(y)])
    axHy.set_xlabel('x')
    axHy.set_ylabel('y')
    axHy.set_title(f'Hy (mode {m})')
    l = LineCollection(edges, colors='black', linewidths=1)
    axHy.add_collection(l)

plt.show()